###  Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
import warnings

### Pass API Key

In [2]:
c = Census("e0577a26a616f4dda60446eae987e3b6d0d944a3")

## Preview Table

In [3]:
cs.printtable(cs.censustable('acs5',2019,'B08303'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08303_001E  | TRAVEL TIME TO WORK            | !! Estimate Total:                                       | int  
B08303_002E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: Less than 5 minutes                | int  
B08303_003E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: 5 to 9 minutes                     | int  
B08303_004E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: 10 to 14 minutes                   | int  
B08303_005E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: 15 to 19 minutes                   | int  
B08303_006E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: 20 to 24 minutes                   | int  
B08303_007E  | TRAVEL TIME TO WORK            | !! !! Estimate Total: 25 to 29 minutes       

In [4]:
Years = [2023, 2022, 2018]
AllData = []

geo = {'for': 'place:16000', 'in': 'state:45'}  # South Carolina places

commute = {
    "B08303_001E": "Total Commuters",
    "B08303_002E": 2.5,
    "B08303_003E": 7,
    "B08303_004E": 12,
    "B08303_005E": 17,
    "B08303_006E": 22,
    "B08303_007E": 27,
    "B08303_008E": 32,
    "B08303_009E": 39.5,
    "B08303_010E": 52,
    "B08303_011E": 74.5,
    "B08303_012E": 95,
}

variables = {
    "B01002_001E": "Median Age",
    "B19013_001E": "Median Household Income",
    "B20017_001E": "Median Individual Income",
    "B25044_001E": "Vehicles Available (Total)",
    "B25044_003E": "Owner: 0 Vehicles",
    "B25044_004E": "Owner: 1 Vehicle",
    "B25044_005E": "Owner: 2 Vehicles",
    "B25044_006E": "Owner: 3 Vehicles",
    "B25044_007E": "Owner: 4+ Vehicles",
    "B25044_010E": "Renter: 0 Vehicles",
    "B25044_011E": "Renter: 1 Vehicle",
    "B25044_012E": "Renter: 2 Vehicles",
    "B25044_013E": "Renter: 3 Vehicles",
    "B25044_014E": "Renter: 4+ Vehicles",
    "B17001_001E": "Total Population (Poverty Table)",
    "B17001_002E": "Below Poverty Level",
    "B25070_007E": "Gross Rent 30-34.9%",
    "B25070_008E": "Gross Rent 35-39.9%",
    "B25070_009E": "Gross Rent 40-49.9%",
    "B25070_010E": "Gross Rent 50%+",
    "B25070_001E": "Total Renters",
}

allvariables = list(variables.keys()) + list(commute.keys())

for year in Years:
    ACS = c.acs5.get(allvariables, geo, year=year)
    Data = pd.DataFrame(ACS)
    Data['Year'] = year

    Data.rename(columns=variables, inplace=True)
    Data.rename(columns={"B08303_001E": "Total Commuters"}, inplace=True)

    columns = list(variables.values()) + ["Total Commuters"] + list(commute.keys())[1:]
    for col in columns:
        if col in Data.columns:
            Data[col] = pd.to_numeric(Data[col], errors='coerce')

    weighted_sum = sum(Data[var] * weight for var, weight in commute.items() if var != "B08303_001E")
    Data["Average Commute Time"] = weighted_sum / Data["Total Commuters"]

    Data["Poverty Rate (%)"] = (
        Data["Below Poverty Level"] / Data["Total Population (Poverty Table)"] * 100
    )

    Data["Cost-Burdened Renters (%)"] = (
        Data[[
            "Gross Rent 30-34.9%", "Gross Rent 35-39.9%",
            "Gross Rent 40-49.9%", "Gross Rent 50%+"
        ]].sum(axis=1) / Data["Total Renters"] * 100
    )

    total_vehicles = (
        0 * (Data["Owner: 0 Vehicles"] + Data["Renter: 0 Vehicles"]) +
        1 * (Data["Owner: 1 Vehicle"] + Data["Renter: 1 Vehicle"]) +
        2 * (Data["Owner: 2 Vehicles"] + Data["Renter: 2 Vehicles"]) +
        3 * (Data["Owner: 3 Vehicles"] + Data["Renter: 3 Vehicles"]) +
        4 * (Data["Owner: 4+ Vehicles"] + Data["Renter: 4+ Vehicles"])
    )
    Data["Average Vehicles per Household"] = total_vehicles / Data["Vehicles Available (Total)"]

    Data["Average Commute Time"] = Data["Average Commute Time"].round(1)
    Data["Average Vehicles per Household"] = Data["Average Vehicles per Household"].round(1)
    Data["Poverty Rate (%)"] = Data["Poverty Rate (%)"].round(1)
    Data["Cost-Burdened Renters (%)"] = Data["Cost-Burdened Renters (%)"].round(1)

    AllData.append(Data)

FinalData = pd.concat(AllData, ignore_index=True)

cols_to_keep = [
    "Year",
    "Median Age", "Median Household Income", "Median Individual Income",
    "Average Commute Time", "Average Vehicles per Household",
    "Poverty Rate (%)", "Cost-Burdened Renters (%)"
]

melted = FinalData[cols_to_keep].melt(id_vars=["Year"], var_name="Metric", value_name="Value")
FinalData = melted.pivot_table(index="Metric", columns="Year", values="Value").reset_index()

FinalData = FinalData[["Metric"] + sorted(Years)]

FinalData


Year,Metric,2018,2022,2023
0,Average Commute Time,17.2,18.0,17.8
1,Average Vehicles per Household,1.5,1.5,1.5
2,Cost-Burdened Renters (%),52.2,52.1,53.5
3,Median Age,28.3,28.3,28.7
4,Median Household Income,45663.0,54095.0,55653.0
5,Median Individual Income,21553.0,28249.0,30016.0
6,Poverty Rate (%),22.5,24.2,23.3


In [5]:
FinalData.to_csv('CityofColumbiaEconomicProfile.csv',index = False)